In [ ]:
import psycopg2


In [ ]:
#The fields below depend on the user's sql set-up for the eICU database
hostname = ###
username = ###
password = ###
database = ###

myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )


In [ ]:
import pandas.io.sql as sqlio

sql = "SELECT patientUnitStayID,observationoffset,sao2 FROM eicu.vitalperiodic ORDER BY patientUnitStayID, observationoffset"
vitals = sqlio.read_sql_query(sql, myConnection)


In [ ]:
#Get these 1326 patients IDs with ventilator
query1="SELECT DISTINCT patientUnitStayID FROM eicu.respiratorycharting WHERE respchartvaluelabel SIMILAR TO '%(ET TUBE|ETT|Endotracheal|Trach|Tracheostomy)%'"
vent_pts=sqlio.read_sql_query(query1, myConnection)




In [ ]:
#Get the non_vent patients by random sample
q2="SELECT * FROM eicu.patient WHERE patientUnitStayID NOT IN (SELECT DISTINCT patientUnitStayID FROM eicu.respiratorycharting WHERE respchartvaluelabel SIMILAR TO '%(ET TUBE|ETT|Endotracheal|Trach|Tracheostomy)%') ORDER BY RANDOM() LIMIT 1326"
not_vent_pts=sqlio.read_sql_query(q2, myConnection)
not_vent_pts.to_csv('RandomNon_vent.csv')


In [ ]:
#get data for ventilator patients
q3="SELECT * FROM eicu.patient WHERE patientUnitStayID IN (SELECT DISTINCT patientUnitStayID FROM eicu.respiratorycharting WHERE respchartvaluelabel SIMILAR TO '%(ET TUBE|ETT|Endotracheal|Trach|Tracheostomy)%')"
vent_pts=sqlio.read_sql_query(q3, myConnection)
#vent_pts.to_csv('AllVent.csv')


In [ ]:
#saving IDs
import pandas as pd
non_vent_pts=pd.read_csv('RandomNon_vent.csv')
non_vent_IDs=non_vent_pts['patientunitstayid'].unique()
vent_pts=pd.read_csv('Allvent.csv')
vent_IDs=vent_pts['patientunitstayid'].unique()

In [ ]:
#getting initial training set
import numpy as np
TrainIDs=np.concatenate((vent_IDs[0:1000],non_vent_IDs[0:1000]))
Vent_TestIDs=vent_IDs[1000:]
non_vent_TestIDs=non_vent_IDs[1000:]
All_IDs=np.concatenate((TrainIDs,Vent_TestIDs,non_vent_TestIDs))

In [ ]:
#imports to work with sql in python
import numpy as np
from psycopg2.extensions import register_adapter, AsIs

def addapt_numpy_array(numpy_array):
    return AsIs(tuple(numpy_array))

register_adapter(np.ndarray, addapt_numpy_array)


In [ ]:
#getting vitals
q="SELECT patientUnitStayID,observationoffset,sao2 FROM eicu.vitalperiodic WHERE patientUnitStayID in %s ORDER BY patientUnitStayID, observationoffset"
vitals=sqlio.read_sql_query(q, myConnection,params=[All_IDs])


In [ ]:
vitals.to_csv('vitals.csv')

In [ ]:
#save the IDs in each set
TrainIDs=np.intersect1d(TrainIDs,vitals['patientunitstayid'].unique())
Test_Vent=np.intersect1d(Vent_TestIDs,vitals['patientunitstayid'].unique())
Test_NoVent=np.intersect1d(non_vent_TestIDs,vitals['patientunitstayid'].unique())


In [ ]:
#get patient information
q="SELECT * FROM eicu.patient WHERE patientUnitStayID in %s ORDER BY patientUnitStayID"
vitals_train=sqlio.read_sql_query(q, myConnection,params=[TrainIDs])
vitals_venttest=sqlio.read_sql_query(q, myConnection,params=[Test_Vent])
vitals_noventtest=sqlio.read_sql_query(q, myConnection,params=[Test_NoVent])


In [ ]:
#uniue patient IDs
#remove patients with stays in both train and test
train_pts=vitals_train['uniquepid']
venttest_pts=vitals_venttest['uniquepid']
noventtest_pts=vitals_noventtest['uniquepid']

In [ ]:
np.intersect1d(train_pts,venttest_pts)

In [ ]:
np.intersect1d(train_pts,noventtest_pts)

In [ ]:
vitals_venttest=vitals_venttest[~vitals_venttest['uniquepid'].isin(['033-18915', '033-21109', '033-28508', '033-3075', '033-35464'])]

In [ ]:
vitals_noventtest=vitals_noventtest[~vitals_noventtest['uniquepid'].isin(['009-15364', '017-30920', '017-84854', '033-41650'])]

In [ ]:
Test_Vent=np.array(vitals_venttest['patientunitstayid'])
Test_NoVent=np.array(vitals_noventtest['patientunitstayid'])

In [ ]:
#save the train, test sets
np.savetxt("TrainEICU.csv", TrainIDs, delimiter=",")
np.savetxt("TestVent.csv", Test_Vent, delimiter=",")
np.savetxt("Test_NoVent.csv", Test_NoVent, delimiter=",")


In [ ]:
Vent_Test_Pts=np.loadtxt("FinalVentTestEICU_30min.csv", delimiter=",")
NonVent_Test_Pts=np.loadtxt("FinalNoVentTestEICU_30min.csv", delimiter=",")
Train_pts=np.loadtxt('TrainEICU.csv')

In [ ]:
#Demographics
#Read in the final lists

q="SELECT * FROM eicu.patient WHERE patientUnitStayID in %s ORDER BY patientUnitStayID"
dems_vent_test=sqlio.read_sql_query(q, myConnection,params=[Vent_Test_Pts])
dems_novent_test=sqlio.read_sql_query(q, myConnection,params=[NonVent_Test_Pts])


In [ ]:
dems_vent_test=dems_vent_test.replace('> 89','90')
medv = dems_vent_test['age'].median()



In [ ]:
dems_novent_test=dems_novent_test.replace('> 89','90')
med_nv=dems_novent_test['age'].median()

In [ ]:
vent_fem = np.sum(dems_vent_test['gender']=='Female')/len(dems_vent_test['gender'])

In [ ]:
nv_fem = np.sum(dems_novent_test['gender']=='Female')/len(dems_novent_test['gender'])

In [ ]:
vent_caucasian = np.sum(dems_vent_test['ethnicity']=='Caucasian')/len(dems_vent_test['ethnicity'])

In [ ]:
nv_caucasian = np.sum(dems_novent_test['ethnicity']=='Caucasian')/len(dems_novent_test['ethnicity'])

In [ ]:
dems_vent_test['apacheadmissiondx'].value_counts()

In [ ]:
dems_novent_test['apacheadmissiondx'].value_counts()